In [1]:
## imports
import pandas as pd
import numpy as np
import xarray as xr
from time import time

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV


In [2]:
## Para guargar el modelo
'''
from joblib import dump, load
dump(clf, 'filename.joblib') # guardar
clf = load('filename.joblib') # cargar
'''

"\nfrom joblib import dump, load\ndump(clf, 'filename.joblib') # guardar\nclf = load('filename.joblib') # cargar\n"

In [3]:
## carga datos pruebas

df_atributos = pd.read_pickle('../pkls/dfAtributosNormalizado_2_dias_2_celdas.pkl')
df_avistamientos = pd.read_pickle('../pkls/dfAvistamientos.pkl')
df_avistamientos.head()
df_atributos.head()

,mlotst_0,zos_0,bottomT_0,thetao_0,so_0,uo_0,vo_0,mlotst_izq_1_0,zos_izq_1_0,bottomT_izq_1_0,...,so_inf_2.1_10,uo_inf_2.1_10,vo_inf_2.1_10,mlotst_inf_2.2_10,zos_inf_2.2_10,bottomT_inf_2.2_10,thetao_inf_2.2_10,so_inf_2.2_10,uo_inf_2.2_10,vo_inf_2.2_10
0,0.028272,0.484909,0.593375,0.393488,0.753866,0.265495,0.377426,0.044560,0.470527,0.578919,...,0.703526,0.535836,0.402959,0.017508,0.430021,0.320409,0.476941,0.670427,0.524089,0.240149
1,0.028272,0.564889,0.748738,0.600051,0.778454,0.335800,0.493170,0.023834,0.563082,0.792167,...,0.755449,0.484642,0.517160,0.015448,0.558761,0.207604,0.861387,0.755555,0.509988,0.422790
2,0.028272,0.430584,0.701932,0.391888,0.746007,0.418131,0.514019,0.022798,0.425026,0.727978,...,0.682051,0.395904,0.635503,0.003090,0.431624,0.305412,0.452228,0.659829,0.545241,0.567625
3,0.028272,0.425553,0.701025,0.380565,0.747275,0.455134,0.485262,0.022798,0.417787,0.729441,...,0.678205,0.408419,0.634911,0.003090,0.422543,0.300898,0.414153,0.655727,0.508813,0.598509
4,0.028272,0.450201,0.694401,0.550601,0.777440,0.284921,0.498922,0.023834,0.450879,0.802019,...,0.732692,0.439135,0.716568,0.006179,0.476496,0.198927,0.749822,0.719316,0.534665,0.571352


In [4]:
atributos = df_atributos
resultado = np.ravel(df_avistamientos)

In [5]:
atributos.shape,resultado.shape

((668, 231), (668,))

In [8]:
## Series temporales
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=10)
# print(tscv)

for train_index, test_index in tscv.split(df_atributos):
#     print("TRAIN:", train_index, "\n\tTEST:", test_index)
    X_train, X_test = df_atributos.iloc[train_index], df_atributos.iloc[test_index]
    y_train, y_test = df_avistamientos.iloc[train_index], df_avistamientos.iloc[test_index]


In [125]:
# tscv = TimeSeriesSplit(n_splits=10)
# forest(atributos,resultado,2,tscv)
# tscv = TimeSeriesSplit(n_splits=5)
# forest(atributos,resultado,2,tscv)
# tscv = TimeSeriesSplit(n_splits=2)
# forest(atributos,resultado,2,tscv)

# Random Forest

In [134]:
## random forest con series temporales

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import randint
def forest(atributos,resultado,n,tscv):
    params = {'n_estimators': randint(1, 200),
               'max_depth': randint(1, 100),
              'max_features': randint(1,atributos.shape[1])}
    inicio = time()
    model_random = RandomizedSearchCV(estimator=RandomForestRegressor(), 
                           cv=tscv,
                           n_iter=n,
                           param_distributions=params,
                                      n_jobs = 4)

    model_random.fit(atributos, resultado)
    fin = time()
    print('{} minutos'.format((fin - inicio)/60), flush=True)
    print('{}\n{}'.format(model_random.best_params_,model_random.best_score_), flush=True)
    return ['{} minutos \n'.format((fin - inicio)/60),model_random.best_params_,model_random.best_score_]

# Nearest Neighbor

In [6]:
## vecino mas cercano con series temporales

from sklearn.neighbors import KNeighborsRegressor
def vecino(atributos,resultado,n,tscv):

    # params = {'n_neighbors' : list(range(1,atributos.shape[1]))}
    params = {'n_neighbors' : list(range(1,60))}

    inicio = time()
    model_vecinos = RandomizedSearchCV(estimator=KNeighborsRegressor(), 
                           cv=tscv,
                           n_iter=n,
                           param_distributions=params,
                                      n_jobs = 4)


    model_vecinos.fit(atributos, resultado)
    fin = time()
    print('{} minutos'.format((fin - inicio)/60))
    print('{}\n{}'.format(model_vecinos.best_params_,model_vecinos.best_score_))
    return ['{} minutos \n'.format((fin - inicio)/60),model_vecinos.best_params_,model_vecinos.best_score_]

In [136]:

def vecino_grid(atributos,resultado,n,tscv):

    # params = {'n_neighbors' : list(range(1,atributos.shape[1]))}
    params = {'n_neighbors' : list(range(1,60))}

    inicio = time()
    
    model_vecinos_grid= GridSearchCV(estimator=KNeighborsRegressor(), 
                           cv=tscv,
                           param_grid=params,
                                    n_jobs = 4)


    model_vecinos_grid.fit(atributos, resultado)
    fin = time()
    print('{} minutos'.format((fin - inicio)/60))
    print('{}\n{}'.format(model_vecinos_grid.best_params_,model_vecinos_grid.best_score_))
    return ['{} minutos \n'.format((fin - inicio)/60),model_vecinos_grid.best_params_,model_vecinos_grid.best_score_]

In [76]:
# model_vecinos.cv_results_


# SVM

In [137]:
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

def SVR(atributos,resultado,n,tscv):

    params = {'svr__kernel':('linear', 'rbf'),'svr__C' : [0.5,1.0,10,100], 'svr__epsilon':[0.2]}

    inicio = time()
    steps =  [('scaler',StandardScaler()), ('svr',svm.SVR())]
    pipeline = Pipeline(steps)


    model_SVR= RandomizedSearchCV(estimator=pipeline, 
                           cv=tscv,
                           n_iter=n,
                           param_distributions=params,
                                 n_jobs = 4)


    model_SVR.fit(atributos, resultado)
    fin = time()
    print('{} minutos'.format((fin - inicio)/60))
    print('{}\n{}'.format(model_SVR.best_params_,model_SVR.best_score_))
    return ['{} minutos \n'.format((fin - inicio)/60),model_SVR.best_params_,model_SVR.best_score_]

In [138]:
# model_SVR.cv_results_

In [139]:
from sklearn import svm
def SVR_grid(atributos,resultadon,n,tscv):
    params = {'svr__kernel':('linear', 'rbf'),'svr__C' : [0.5,1.0,10,100], 'svr__epsilon':[0.2]}

    inicio = time()
    steps =  [('scaler',StandardScaler()), ('svr',svm.SVR())]
    pipeline = Pipeline(steps)

    model_SVR_grid= GridSearchCV(estimator=pipeline, 
                           cv=tscv,
                           param_grid=params,
                            n_jobs = 4)


    model_SVR_grid.fit(atributos, resultado)
    fin = time()
    print('{} minutos'.format((fin - inicio)/60))
    print('{}\n{}'.format(model_SVR_grid.best_params_,model_SVR_grid.best_score_))
    return ['{} minutos \n'.format((fin - inicio)/60),model_SVR_grid.best_params_,model_SVR_grid.best_score_]

In [80]:
# model_SVR_grid.cv_results_

# Árboles de decisión

In [140]:
from sklearn.tree import DecisionTreeRegressor
def arbol_decision(atributos,resultadon,n,tscv):
    params = {'max_depth':list(range(1,50)),'max_features' : ['auto', 'sqrt', 'log2',None]}

    inicio = time()
    model_tree= RandomizedSearchCV(estimator=DecisionTreeRegressor(), 
                           cv=tscv,
                           n_iter=n,
                           param_distributions=params,
                            n_jobs = 4)


    model_tree.fit(atributos, resultado)
    fin = time()
    print('{} minutos'.format((fin - inicio)/60))
    print('{}\n{}'.format(model_tree.best_params_,model_tree.best_score_))
    return ['{} minutos \n'.format((fin - inicio)/60),model_tree.best_params_,model_tree.best_score_]

In [141]:
def arbol_decision_grid(atributos,resultadon,n,tscv):
    params = {'max_depth':list(range(1,50)),'max_features' : ['auto', 'sqrt', 'log2',None]}

    inicio = time()
    model_tree= GridSearchCV(estimator=DecisionTreeRegressor(), 
                           cv=tscv,
                           param_grid=params,
                            n_jobs = 4)


    model_tree.fit(atributos, resultado)
    fin = time()
    print('{} minutos'.format((fin - inicio)/60))
    print('{}\n{}'.format(model_tree.best_params_,model_tree.best_score_))
    return ['{} minutos \n'.format((fin - inicio)/60),model_tree.best_params_,model_tree.best_score_]

In [83]:
# model_tree.cv_results_

# Red Neuronal

## MLP

In [145]:
# from sklearn.neural_network import MLPRegressor

# params = {'mlp__alpha':('linear', 'rbf'),
#           'mlp__max_iter' : [200,500,2000],
#           'mlp__random_state': [0,1,10]}

# inicio = time()

# steps =  [('scaler',StandardScaler()), ('mlp',MLPRegressor())]
# pipeline = Pipeline(steps)

# model_MLP= GridSearchCV(estimator=pipeline, 
#                        cv=tscv,
#                        param_grid=params)


# model_MLP.fit(atributos, resultado)
# fin = time()
# print('{} minutos'.format((fin - inicio)/60))

In [36]:
atributos.describe()

,mlotst_0,zos_0,bottomT_0,thetao_0,so_0,uo_0,vo_0,mlotst_izq_1_0,zos_izq_1_0,bottomT_izq_1_0,...,so_inf_2.1_10,uo_inf_2.1_10,vo_inf_2.1_10,mlotst_inf_2.2_10,zos_inf_2.2_10,bottomT_inf_2.2_10,thetao_inf_2.2_10,so_inf_2.2_10,uo_inf_2.2_10,vo_inf_2.2_10
count,668.000000,668.000000,668.000000,668.000000,668.000000,668.000000,668.000000,668.000000,668.000000,668.000000,...,668.000000,668.000000,668.000000,668.000000,668.000000,668.000000,668.000000,668.000000,668.000000,668.000000
mean,0.041073,0.309653,0.610348,0.334054,0.664455,0.432030,0.504365,0.048621,0.301016,0.666412,...,0.577418,0.527978,0.578173,0.038609,0.285635,0.440186,0.348443,0.545809,0.528888,0.505152
std,0.065048,0.119846,0.093025,0.150555,0.113130,0.132660,0.163475,0.072797,0.118568,0.107568,...,0.126843,0.140078,0.183272,0.082772,0.114482,0.224055,0.173238,0.134399,0.131468,0.164769
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.028272,0.242958,0.555126,0.238757,0.632383,0.355227,0.390906,0.023834,0.234100,0.635743,...,0.525561,0.439135,0.511391,0.004119,0.222756,0.237649,0.224914,0.493675,0.446533,0.447018
50%,0.028272,0.304829,0.602721,0.323183,0.670596,0.418131,0.513659,0.023834,0.298862,0.682275,...,0.573558,0.526166,0.612426,0.004119,0.288996,0.446807,0.340598,0.545299,0.531140,0.527157
75%,0.032461,0.378898,0.676341,0.412231,0.745373,0.505319,0.601546,0.036528,0.372285,0.717430,...,0.673478,0.613481,0.685207,0.028836,0.351496,0.661082,0.432980,0.646837,0.614571,0.599707
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Boosting

In [132]:
from sklearn.ensemble import GradientBoostingRegressor
def boosting(atributos,resultadon,n,tscv):
    params = {'n_estimators': randint(1, 200),
          'max_depth': randint(1, 100)}

    inicio = time()
    model_boos= RandomizedSearchCV(estimator=GradientBoostingRegressor(), 
                           cv=tscv,
                           n_iter=n,
                           param_distributions=params,
                           n_jobs = 4)


    model_boos.fit(atributos, resultado)
    fin = time()
    print('{} minutos'.format((fin - inicio)/60))
    print('{}\n{}'.format(model_boos.best_params_,model_boos.best_score_))
    return ['{} minutos \n'.format((fin - inicio)/60),model_boos.best_params_,model_boos.best_score_]

In [142]:
# boosting(atributos,resultado,2,tscv) 

# ensembles

In [115]:
# df para guardar resultados
algoritmos  = ['random_forest','nearest_neighbor','nearest_neighbor_grid',
               'SVR','SVR_grid','arbol_decision', 'arbol_decision_grid',
               'MLP','Boosting','ensembles']
df = pd.DataFrame(index=algoritmos)

def reinicia_df():
    df = pd.DataFrame(index=algoritmos)
    df.to_pickle('resultados.pkl')

def guarda_resultado(alg,dias,celdas,split,resultado,params):
    df = pd.read_pickle('resultados.pkl')
    print(alg,dias,celdas,resultado,params)
    nombre_col = '{}_dias_{}_celdas_{}_splits'.format(dias,celdas,split)
    if not nombre_col in df.columns:
        df[nombre_col] = np.nan
        df[nombre_col + '_params'] = np.nan
    df.loc[alg,nombre_col] = resultado
    df.loc[alg,nombre_col + '_params'] = params
    df.to_pickle('resultados.pkl')
    
guarda_resultado('random_forest',1,2,0,3,'hola')
guarda_resultado('nearest_neighbor',1,2,0,4,'adios')
guarda_resultado('SVR',2,2,0,4,'salu2')
df 

random_forest 1 2 3 hola
nearest_neighbor 1 2 4 adios
SVR 2 2 4 salu2


""
random_forest
nearest_neighbor
nearest_neighbor_grid
SVR
SVR_grid
arbol_decision
arbol_decision_grid
MLP
Boosting
ensembles


In [116]:
a = pd.read_pickle('resultados.pkl')
a

,1_dias_2_celdas_0_splits,1_dias_2_celdas_0_splits_params,2_dias_2_celdas_0_splits,2_dias_2_celdas_0_splits_params
random_forest,3.0,hola,NaN,NaN
nearest_neighbor,4.0,adios,NaN,NaN
nearest_neighbor_grid,NaN,NaN,NaN,NaN
SVR,NaN,NaN,4.0,salu2
SVR_grid,NaN,NaN,NaN,NaN
arbol_decision,NaN,NaN,NaN,NaN
arbol_decision_grid,NaN,NaN,NaN,NaN
MLP,NaN,NaN,NaN,NaN
Boosting,NaN,NaN,NaN,NaN
ensembles,NaN,NaN,NaN,NaN


In [ ]:
import os
from sklearn.model_selection import TimeSeriesSplit
import datetime

splits = [2,5,10]

res = ''
reinicia_df()
avistamientos = pd.read_pickle('../pkls/dfAvistamientos.pkl')
resultado = np.ravel(df_avistamientos)

listado_archivos = os.listdir('../pkls/')
df = pd.DataFrame(index=algoritmos)
for i in listado_archivos:
    for split in splits:
        tscv = TimeSeriesSplit(n_splits=split)
        if 'dfAtributosNormalizado' in i :
            atributos = pd.read_pickle('../pkls/{}'.format(i))
            dias,celdas = i[23:25],i[31:33]
            n_iter = 100
            print('\n\n' + i + '\n', flush=True)

            # concateno y guardo en log
            res += str(datetime.datetime.now()) + ' --------- ' +  str(i) + '\n'

            model_result = forest(atributos,resultado,n_iter,tscv)
            guarda_resultado('random_forest',dias,celdas,split,model_result[2],str(model_result[1]))
            res += '--> Random Forest ' + str(model_result) + '\n'

            model_result = vecino(atributos,resultado,n_iter,tscv)
            guarda_resultado('nearest_neighbor',dias,celdas,split,model_result[2],str(model_result[1]))
            res += '--> Vecino mas cercano ' + str(model_result) + '\n'

            model_result = vecino_grid(atributos,resultado,n_iter,tscv)
            guarda_resultado('nearest_neighbor_grid',dias,celdas,split,model_result[2],str(model_result[1]))
            res += '--> Vecino mas cercano gridSearch ' + str(model_result) + '\n'

            model_result = SVR(atributos,resultado,n_iter,tscv)
            guarda_resultado('SVR',dias,celdas,split,model_result[2],str(model_result[1]))
            res += '--> SVR ' + str(model_result) + '\n'

            model_result = SVR_grid(atributos,resultado,n_iter,tscv)
            guarda_resultado('SVR_grid',dias,celdas,split,model_result[2],str(model_result[1]))
            res += '--> SVR_grid ' + str(model_result) + '\n'

            model_result = arbol_decision(atributos,resultado,n_iter,tscv)
            guarda_resultado('arbol_decision',dias,celdas,split,model_result[2],str(model_result[1]))
            res += '--> arbol_decision ' + str(model_result) + '\n'

            model_result = arbol_decision_grid(atributos,resultado,n_iter,tscv)
            guarda_resultado('arbol_decision_grid',dias,celdas,split,model_result[2],str(model_result[1]))
            res += '--> arbol_decision_grid ' + str(model_result) + '\n'
            
            model_result = boosting(atributos,resultado,n_iter,tscv)
            guarda_resultado('Boosting',dias,celdas,split,model_result[2],str(model_result[1]))
            res += '--> Boosting ' + str(model_result) + '\n'

        f = open ('log.txt','wb')

        f.write(bytes(res, encoding='utf-8'))
        f.close()

In [97]:
## Regresion Lineal

df

""
random_forest
nearest_neighbor
nearest_neighbor_grid
SVR
SVR_grid
arbol_decision
arbol_decision_grid
MLP
Boosting
ensembles


In [113]:
tscv = TimeSeriesSplit(n_splits=2)

from sklearn.tree import DecisionTreeRegressor


params = {'max_depth':[381],'max_features' : ['sqrt']}

inicio = time()
model_tree= GridSearchCV(estimator=DecisionTreeRegressor(), 
                       cv=tscv,
                       param_grid=params,
                        n_jobs = -1)


model_tree.fit(atributos, resultado)
fin = time()
print('{} minutos'.format((fin - inicio)/60))
print('{}\n{}'.format(model_tree.best_params_,model_tree.best_score_))

0.0005670428276062012 minutos
{'max_depth': 381, 'max_features': 'sqrt'}
-1.326596210342386


In [141]:
tscv = TimeSeriesSplit(n_splits=2)
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor

lista = [TimeSeriesSplit(n_splits=2),TimeSeriesSplit(n_splits=5),TimeSeriesSplit(n_splits=10)]
models = []

for i in lista :
    
    params = {'max_depth':list(range(1,1000)),'max_features' : ['auto', 'sqrt', 'log2',None]}

    inicio = time()
    model_tree= GridSearchCV(estimator=DecisionTreeRegressor(), 
                           cv=i,
                           param_grid=params,
                            n_jobs = -1)


    model_tree.fit(atributos, resultado)
    models.append(model_tree)
    fin = time()
    print('{} minutos'.format((fin - inicio)/60))
    print('{}\n{}'.format(model_tree.best_params_,model_tree.best_score_))
    # return ['{} minutos \n'.format((fin - inicio)/60),model_tree.best_params_,model_tree.best_score_]

0.5677285353342693 minutos
{'max_depth': 223, 'max_features': 'log2'}
0.19443376814711577
1.3161501208941142 minutos
{'max_depth': 2, 'max_features': 'sqrt'}
-0.15865642250286363
2.632586375872294 minutos
{'max_depth': 616, 'max_features': 'sqrt'}
-0.5232034053092856


In [139]:
from joblib import dump, load
dump(model_tree, 'modelo023.joblib') 


['modelo023.joblib']

In [106]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4, 5, 6])
tscv = TimeSeriesSplit(n_splits=2)
print(tscv)

for train_index, test_index in tscv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


TimeSeriesSplit(max_train_size=None, n_splits=2)
TRAIN: [0 1] TEST: [2 3]
TRAIN: [0 1 2 3] TEST: [4 5]


In [20]:

# params = {'n_neighbors' : list(range(1,atributos.shape[1]))}
params = {'n_neighbors' : list(range(1,50))}

inicio = time()
tscv = TimeSeriesSplit(n_splits=10)

model_vecinos_grid= GridSearchCV(estimator=KNeighborsRegressor(), 
                       #cv=tscv,
                       param_grid=params,
                       n_jobs = -1)




In [13]:
model_vecinos_grid.fit(atributos, resultado)
fin = time()
print('{} minutos'.format((fin - inicio)/60))
print('{}\n{}'.format(model_vecinos_grid.best_params_,model_vecinos_grid.best_score_))


0.02162463665008545 minutos
{'n_neighbors': 9}
-3.6456533631383268


In [21]:
from sklearn.model_selection import cross_val_score

cross_val_score(model_vecinos_grid,atributos,resultado)

array([-4.02741075,  0.1053884 ,  0.15046699, -8.63113957, -0.04849008])